In [379]:
import random as rnd
import re
    

def task_rsa():
    with open('russian_nouns.txt', mode='r', encoding='utf8') as f:
        words = [word.strip() for word in f]

    words4 = [word for word in words if len(word) == 4]
    primes = [5, 7, 11, 13, 17]
    
    while True:
        rnd.shuffle(primes)
        p = primes[0]
        q = primes[1]
        n = p * q
        phi = euler_phi(n)
        e = rnd.randint(2, phi - 2)
        if gcd(e, phi) != 1:
            continue
        R_phi = IntegerModRing(phi)
        d = R_phi(1) / R_phi(e)
        
        random_word = words4[rnd.randint(0, len(words4) - 1)]
        codes = [ord(letter) - ord('а') + 2 for letter in random_word]
        encoded_codes = [code ^ e % n for code in codes]
        
        def have_similar_codes(list1, list2):
            for a, b in zip(list1, list2):
                if a == b:
                    return True
            return False
    
        
        if have_similar_codes(codes, encoded_codes):
            continue
            
        codes_text = ','.join(str(code) for code in encoded_codes)
        
        yield f'''Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
        на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
        от других букв.
        Шифрованое сообщение: $({codes_text})$, открытый ключ $({n}, {e})$. Восстановите закрытый ключ и
        расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.'''
        
def task_haffman(symbols_count=6):
    with open('russian_nouns.txt', mode='r', encoding='utf8') as f:
        words = [word.strip() for word in f]

    words = [word for word in words if len(word) == symbols_count and len(set(word)) == symbols_count]
    
    while True:
        random_word = words[rnd.randint(0, len(words) - 1)]
        random_frequencies = rnd.choices(range(100), k=symbols_count)
        
        symbols = sorted(zip(random_frequencies, random_word))
        process = symbols.copy()
        wrong_counts = False
        for _ in range(symbols_count - 2):
            c1, s1 = process.pop(0)
            c2, s2 = process.pop(0)
            c3, _ = process[0]
            if c1 == c2 or c2 == c3:
                wrong_counts = True
                break
            process.insert(0, (c1 + c2, s1 + s2))
            process.sort()
        if wrong_counts:
            continue
        if len(process[0][1]) == 1 or len(process[1][1]) == 1:
            continue
            
        frequencies_texts = ', '.join(f'`{s}`: {c}' for c, s in symbols)
            
        yield f'''Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
        {frequencies_texts}. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.'''
        
        
def task_permutation_by_number(n=7):
    number = rnd.randint(42, factorial(n) - 42)
    while True:
        yield f'''Если упорядочить все перестановки чисел от 1 до {n}, какая перестановка идет под номером {number}?'''
        
def task_permutation_number(n=7):
    numbers = list(range(1, n + 1))
    
    while True:
        rnd.shuffle(numbers)
        if numbers[0] == n or numbers[0] == 1:
            continue
        permutation = ''.join(str(a) for a in numbers)
        yield f'''Если упорядочить все перестановки чисел от 1 до {n}, какой по счету будет перестановка {permutation}?'''
    
    
def task_polygcd(deg=4, gcd_deg=2):
    x = var('x')
    poly_ring = QQ[x]
    coefs_variants = [-3, -2, -1, 1, 2, 3]
    
    while True:
        gcd_coefs = rnd.choices(coefs_variants, k=gcd_deg + 1)
        mul1_coefs = rnd.choices(coefs_variants, k=deg - gcd_deg + 1)
        mul2_coefs = rnd.choices(coefs_variants, k=deg - gcd_deg + 1)
        def coefs_to_poly(coefs):
            return poly_ring(sum(coef * x^i for i, coef in enumerate(coefs)))

        gcd_poly = coefs_to_poly(gcd_coefs)
        mul1_poly = coefs_to_poly(mul1_coefs)
        mul2_poly = coefs_to_poly(mul2_coefs)

        p1 = gcd_poly * mul1_poly
        p2 = gcd_poly * mul2_poly

        if gcd(p1, p2).degree() > gcd_deg:
            continue

        yield fr'''Вычислите НОД многочленов ${latex(p1)}$ и ${latex(p2)}$ над ''' + r'$\mathbf{R}[x]$'
        
            
def task_polygcd_mod(deg=4, gcd_deg=2, mods=(3, 5, 7)):
    x = var('x')
    mod = rnd.choice(mods)
    poly_ring = IntegerModRing(mod)[x]
    coefs_variants = list(range(1, mod))
    
    #TODO dry
    while True:
        gcd_coefs = rnd.choices(coefs_variants, k=gcd_deg + 1)
        mul1_coefs = rnd.choices(coefs_variants, k=deg - gcd_deg + 1)
        mul2_coefs = rnd.choices(coefs_variants, k=deg - gcd_deg + 1)
        def coefs_to_poly(coefs):
            return poly_ring(sum(coef * x^i for i, coef in enumerate(coefs)))

        gcd_poly = coefs_to_poly(gcd_coefs)
        mul1_poly = coefs_to_poly(mul1_coefs)
        mul2_poly = coefs_to_poly(mul2_coefs)

        p1 = gcd_poly * mul1_poly
        p2 = gcd_poly * mul2_poly

        if gcd(p1, p2).degree() > gcd_deg:
            continue

        yield f'''Вычислите НОД многочленов ${latex(p1)}$ и ${latex(p2)}$ над $\\mathbf{{Z}}_{mod}[x]$'''
    
def task_include_exclude(divs_count=4):
    while True:
        n = rnd.randint(1000, 9999)
        divs = sorted(rnd.choices(range(2, 20), k=divs_count))
        wrong_division = False
        for i in range(divs_count):
            for j in range(i + 1, divs_count):
                if divs[j] % divs[i] == 0:
                    wrong_division = True
        if wrong_division:
            continue    
            
        divs_text = ', '.join(str(div) for div in divs)

        yield f'''Сколько чисел от $1$ до ${n}$ делится хотя бы на одно из этих чисел: ${{{divs_text}}}$?'''

In [380]:
from IPython.display import Markdown, display


def pad_text(text, level=1):
    pad = "  " * level
    lines = [pad + line.strip() for line in text.split('\n')]
    return '\n'.join(lines)


def generate_tasks(tasks, variants_number, seed=None, seeds=None):
    if seeds is None:
        if seed is None:
            seeds = rnd.choices(range(2**63 - 1), k=len(tasks))
        else:
            seeds = [seed] * len(tasks)
            
    tasks_states = []
    for seed in seeds:
        if isinstance(seed, Integer):
            seed = int(seed)
        rnd.seed(seed)
        tasks_states.append(rnd.getstate())
    
    md = []
    for variant in range(1, variants_number + 1):
        md.append(f'\n## Вариант {variant}')
        
        next_tasks_states = []
        
        for task, state in zip(tasks, tasks_states):
            rnd.setstate(state)
            task_text = '1. ' + pad_text(next(task))
            md.append(task_text)
            next_tasks_states.append(rnd.getstate())
            
        tasks_states = next_tasks_states
        
    return Markdown('\n'.join(md))

In [381]:
text = generate_tasks([
    task_rsa(),
    task_haffman(),
    task_permutation_by_number(),
    task_permutation_number(),
    task_polygcd(),
    task_polygcd_mod(),
    task_include_exclude()
], 30, seed=123)
display(text)


## Вариант 1
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(91,98,53,60)$, открытый ключ $(119, 23)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `у`: 0, `м`: 37, `т`: 73, `з`: 77, `и`: 87, `а`: 90. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 6524731?
1.   Вычислите НОД многочленов $3 x^{4} + 6 x^{3} + 3 x^{2} + 9$ и $-3 x^{4} - 7 x^{3} - 4 x^{2} + 3 x - 3$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $x^{4} + x^{3} + x^{2} + 2$ и $x^{4} + 2 x^{2} + 2 x + 2$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $9400$ делится хотя бы на одно из этих чисел: ${9, 11, 12, 16}$?

## Вариант 2
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(141,140,152,50)$, открытый ключ $(221, 101)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `к`: 2, `у`: 9, `л`: 14, `я`: 79, `г`: 90, `а`: 91. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 6724513?
1.   Вычислите НОД многочленов $x^{4} + 4 x^{3} + 8 x^{2} + 11 x + 6$ и $3 x^{4} + 2 x^{3} + 11 x^{2} + 9$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $2 x^{4} + x^{2} + x + 1$ и $x^{4} + 2 x^{3} + x^{2} + 2$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $6765$ делится хотя бы на одно из этих чисел: ${9, 11, 16, 19}$?

## Вариант 3
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(80,71,63,31)$, открытый ключ $(91, 53)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `и`: 3, `й`: 30, `б`: 46, `у`: 47, `а`: 48, `ц`: 52. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 5321467?
1.   Вычислите НОД многочленов $6 x^{4} - 11 x^{3} - 10 x^{2} + 9 x + 6$ и $3 x^{4} + 8 x^{3} - 14 x^{2} - 3 x + 6$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $2 x^{4} + 2 x^{2} + 2$ и $2 x^{4} + x^{3} + x + 2$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $5200$ делится хотя бы на одно из этих чисел: ${13, 14, 16, 19}$?

## Вариант 4
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(25,16,9,74)$, открытый ключ $(77, 49)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `е`: 1, `о`: 56, `ф`: 63, `ь`: 64, `л`: 89, `р`: 96. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 3461275?
1.   Вычислите НОД многочленов $-4 x^{4} - 2 x^{3} + 8 x^{2} - 7 x + 2$ и $4 x^{4} + 8 x^{3} + x^{2} + x - 2$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $x^{4} + 2 x^{3} + 2 x^{2} + x + 1$ и $x^{4} + 2 x^{3} + x^{2} + 2$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $7865$ делится хотя бы на одно из этих чисел: ${3, 4, 14, 17}$?

## Вариант 5
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(58,72,43,75)$, открытый ключ $(85, 31)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `и`: 8, `г`: 48, `п`: 51, `д`: 64, `в`: 72, `о`: 84. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 6152473?
1.   Вычислите НОД многочленов $2 x^{4} + 3 x^{3} - 2 x^{2} - 5 x - 2$ и $-2 x^{4} + 7 x^{3} + x^{2} - 7 x - 2$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $2 x^{4} + 2$ и $2 x^{4} + x^{3} + 2 x^{2} + 1$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $6016$ делится хотя бы на одно из этих чисел: ${4, 13, 15, 18}$?

## Вариант 6
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(76,166,173,56)$, открытый ключ $(221, 59)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `о`: 1, `а`: 36, `ф`: 58, `с`: 83, `й`: 89, `д`: 99. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 3675412?
1.   Вычислите НОД многочленов $-6 x^{4} + 4 x^{3} - 6 x + 4$ и $6 x^{4} - x^{3} - 14 x^{2} - x + 6$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $2 x^{4} + 2 x^{2} + 2$ и $2 x^{4} + x^{2} + 2 x + 1$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $1337$ делится хотя бы на одно из этих чисел: ${8, 15, 17, 19}$?

## Вариант 7
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(12,2,13,20)$, открытый ключ $(55, 23)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `п`: 53, `в`: 63, `р`: 72, `о`: 90, `л`: 96, `а`: 99. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 2671453?
1.   Вычислите НОД многочленов $3 x^{4} - 2 x^{3} + 3 x^{2} + 1$ и $-3 x^{4} + 8 x^{3} + 8 x^{2} + 5 x + 2$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $2 x^{4} + 2 x^{2} + 2$ и $2 x^{4} + x^{3} + x + 2$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $2316$ делится хотя бы на одно из этих чисел: ${11, 15, 18, 19}$?

## Вариант 8
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(59,60,63,46)$, открытый ключ $(65, 31)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `у`: 15, `а`: 37, `т`: 53, `и`: 54, `ц`: 72, `к`: 98. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 6237154?
1.   Вычислите НОД многочленов $-6 x^{4} - 7 x^{3} - 13 x^{2} - 7 x - 3$ и $-6 x^{4} - 9 x^{3} - 14 x^{2} - 10 x - 3$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $2 x^{4} + 2$ и $x^{4} + 2 x^{3} + x^{2} + 2$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $8627$ делится хотя бы на одно из этих чисел: ${7, 12, 13, 19}$?

## Вариант 9
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(26,53,73,73)$, открытый ключ $(77, 29)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `р`: 4, `ы`: 13, `с`: 19, `и`: 25, `а`: 27, `х`: 35. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 2476513?
1.   Вычислите НОД многочленов $x^{4} - x^{3} - 5 x^{2} + 7 x - 2$ и $3 x^{4} + 9 x^{3} + x^{2} - 7 x + 2$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $x^{4} + 2 x^{3} + 2 x^{2} + x + 1$ и $x^{4} + 1$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $5356$ делится хотя бы на одно из этих чисел: ${8, 12, 17, 19}$?

## Вариант 10
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(24,33,47,11)$, открытый ключ $(65, 11)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `а`: 0, `л`: 8, `д`: 51, `з`: 61, `р`: 63, `е`: 70. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 4132756?
1.   Вычислите НОД многочленов $x^{4} + 3 x^{3} + 6 x^{2} + 6 x + 4$ и $x^{4} + 5 x^{3} + 11 x^{2} + 12 x + 6$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $x^{4} + 2 x^{3} + x^{2} + 2$ и $x^{4} + 1$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $3281$ делится хотя бы на одно из этих чисел: ${3, 4, 10, 14}$?

## Вариант 11
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(171,135,158,117)$, открытый ключ $(187, 37)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `е`: 2, `я`: 32, `х`: 37, `и`: 54, `н`: 63, `а`: 79. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 6217534?
1.   Вычислите НОД многочленов $-3 x^{4} + 5 x^{3} - 3 x + 1$ и $-2 x^{4} + 5 x^{3} - 7 x^{2} + 7 x - 3$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $2 x^{4} + 2 x^{3} + 2 x + 2$ и $x^{4} + x^{3} + x^{2} + 2$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $4495$ делится хотя бы на одно из этих чисел: ${2, 9, 11, 17}$?

## Вариант 12
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(24,41,82,4)$, открытый ключ $(91, 47)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `ц`: 5, `ь`: 14, `е`: 51, `л`: 53, `б`: 65, `а`: 91. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 5164237?
1.   Вычислите НОД многочленов $6 x^{4} - 3 x^{3} - 3 x^{2} + 9 x - 6$ и $6 x^{4} + x^{3} - 9 x^{2} + x + 2$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $2 x^{4} + x^{2} + x + 1$ и $2 x^{4} + 2 x^{3} + 2 x^{2} + 1$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $9898$ делится хотя бы на одно из этих чисел: ${6, 8, 9, 19}$?

## Вариант 13
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(117,74,133,117)$, открытый ключ $(187, 29)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `л`: 11, `у`: 16, `с`: 33, `п`: 36, `о`: 53, `х`: 58. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 4531276?
1.   Вычислите НОД многочленов $6 x^{4} + 3 x^{3} - 8 x^{2} - 6 x - 1$ и $4 x^{4} + 8 x^{3} + 7 x^{2} + 4 x + 1$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $2 x^{4} + 2 x^{3} + 2 x + 2$ и $2 x^{4} + 2 x^{3} + 2 x^{2} + 1$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $6869$ делится хотя бы на одно из этих чисел: ${9, 10, 11, 13}$?

## Вариант 14
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(177,94,52,63)$, открытый ключ $(187, 39)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `р`: 6, `а`: 13, `б`: 34, `з`: 66, `л`: 74, `е`: 87. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 5642137?
1.   Вычислите НОД многочленов $-9 x^{4} - 6 x^{3} - 6 x^{2} - 5 x - 2$ и $-6 x^{4} - 15 x^{3} - 17 x^{2} - 9 x - 2$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $2 x^{4} + x^{3} + 2 x^{2} + 1$ и $2 x^{4} + 2 x^{2} + 2$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $2955$ делится хотя бы на одно из этих чисел: ${4, 7, 13, 18}$?

## Вариант 15
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(32,73,32,37)$, открытый ключ $(85, 37)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `п`: 15, `а`: 32, `к`: 42, `э`: 45, `ж`: 51, `и`: 77. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 6324751?
1.   Вычислите НОД многочленов $-3 x^{4} - 7 x^{3} + x^{2} - 4 x - 2$ и $-3 x^{4} - x^{3} + 9 x^{2} - 8 x + 6$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $x^{4} + x^{3} + x + 1$ и $2 x^{4} + 2 x^{2} + 2$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $1877$ делится хотя бы на одно из этих чисел: ${5, 7, 11, 13}$?

## Вариант 16
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(27,76,184,76)$, открытый ключ $(221, 91)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `т`: 13, `а`: 34, `и`: 36, `о`: 50, `р`: 54, `ц`: 89. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 2375146?
1.   Вычислите НОД многочленов $2 x^{4} - 7 x^{3} + 7 x^{2} - 7 x + 2$ и $-3 x^{4} + 10 x^{3} - 4 x^{2} - 5 x + 2$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $x^{4} + 2 x^{3} + x^{2} + 2$ и $2 x^{4} + x^{2} + 2 x + 1$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $4542$ делится хотя бы на одно из этих чисел: ${2, 3, 7, 11}$?

## Вариант 17
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(40,81,73,52)$, открытый ключ $(85, 47)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `е`: 4, `и`: 24, `о`: 26, `д`: 50, `р`: 88, `м`: 92. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 4732615?
1.   Вычислите НОД многочленов $4 x^{4} - 2 x^{3} - 6 x^{2} - 15 x - 9$ и $4 x^{4} + 10 x^{3} + 10 x^{2} + 7 x - 3$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $x^{4} + 2 x^{3} + x^{2} + 2$ и $x^{4} + 2 x^{2} + x + 2$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $1511$ делится хотя бы на одно из этих чисел: ${2, 5, 11, 17}$?

## Вариант 18
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(152,76,197,140)$, открытый ключ $(221, 35)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `д`: 2, `и`: 39, `с`: 49, `е`: 71, `н`: 82, `ь`: 86. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 6452317?
1.   Вычислите НОД многочленов $6 x^{4} + 5 x^{3} + 5 x^{2} + x - 2$ и $6 x^{4} + 8 x^{3} - 3 x^{2} - 3 x + 1$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $x^{4} + x^{3} + x^{2} + 2$ и $2 x^{4} + 2$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $5656$ делится хотя бы на одно из этих чисел: ${13, 15, 17, 19}$?

## Вариант 19
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(60,65,64,12)$, открытый ключ $(85, 19)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `д`: 12, `а`: 22, `п`: 39, `н`: 50, `у`: 63, `с`: 89. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 4216753?
1.   Вычислите НОД многочленов $-2 x^{4} - 5 x^{3} + x^{2} + 2$ и $x^{4} - x^{3} - 7 x^{2} + 8 x - 2$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $x^{4} + x^{3} + x^{2} + 2$ и $x^{4} + 1$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $7102$ делится хотя бы на одно из этих чисел: ${13, 15, 18, 19}$?

## Вариант 20
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(31,41,54,19)$, открытый ключ $(77, 53)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `а`: 5, `с`: 12, `и`: 49, `ф`: 60, `к`: 69, `ж`: 86. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 2145673?
1.   Вычислите НОД многочленов $-3 x^{4} - 4 x^{3} - 6 x^{2} + 4 x - 3$ и $-x^{4} + x^{3} + 3 x - 9$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $x^{4} + 2 x^{3} + 2 x + 1$ и $2 x^{4} + 2 x^{2} + 2$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $2629$ делится хотя бы на одно из этих чисел: ${6, 10, 11, 15}$?

## Вариант 21
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(96,85,134,28)$, открытый ключ $(143, 49)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `т`: 21, `а`: 31, `ё`: 44, `ж`: 81, `л`: 85, `п`: 95. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 6314275?
1.   Вычислите НОД многочленов $-2 x^{4} - x^{3} + 3 x^{2} - 4 x + 4$ и $-3 x^{4} - x^{3} + 6 x^{2} - 6 x + 4$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $x^{4} + x^{3} + x^{2} + 2$ и $x^{4} + 2 x^{2} + x + 2$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $9605$ делится хотя бы на одно из этих чисел: ${7, 10, 12, 17}$?

## Вариант 22
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(31,57,33,38)$, открытый ключ $(65, 23)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `е`: 6, `о`: 18, `л`: 41, `ь`: 50, `к`: 82, `б`: 97. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 2753641?
1.   Вычислите НОД многочленов $x^{4} - 3 x^{3} + x^{2} + 4 x - 6$ и $-3 x^{4} + 6 x^{3} + 5 x^{2} - 8 x + 3$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $2 x^{4} + 2 x^{3} + 2 x^{2} + 1$ и $x^{4} + 2 x^{2} + x + 2$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $5381$ делится хотя бы на одно из этих чисел: ${6, 8, 10, 15}$?

## Вариант 23
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(24,18,7,41)$, открытый ключ $(55, 37)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `м`: 6, `е`: 29, `а`: 37, `ь`: 50, `к`: 85, `д`: 90. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 5237461?
1.   Вычислите НОД многочленов $2 x^{4} + 4 x^{3} - 10 x^{2} + 12 x - 6$ и $-6 x^{4} + 4 x^{3} + 2 x^{2} - 8 x + 6$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $2 x^{4} + 2 x^{2} + 2$ и $x^{4} + 2 x^{2} + x + 2$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $3298$ делится хотя бы на одно из этих чисел: ${4, 9, 13, 14}$?

## Вариант 24
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(10,81,25,25)$, открытый ключ $(85, 23)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `о`: 24, `е`: 34, `к`: 44, `с`: 56, `и`: 84, `л`: 94. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 5712364?
1.   Вычислите НОД многочленов $4 x^{4} - 4 x^{3} + 7 x^{2} - 3 x + 2$ и $-6 x^{4} + x^{3} - 9 x^{2} - 4$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $x^{4} + x^{2} + 1$ и $2 x^{4} + x^{3} + x + 2$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $3929$ делится хотя бы на одно из этих чисел: ${11, 13, 15, 17}$?

## Вариант 25
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(60,63,46,69)$, открытый ключ $(77, 43)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `а`: 1, `ц`: 18, `щ`: 27, `р`: 50, `о`: 90, `и`: 95. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 6247315?
1.   Вычислите НОД многочленов $-4 x^{4} + 4 x^{3} + 5 x^{2} - 11 x + 6$ и $-6 x^{4} + 15 x^{3} - 13 x^{2} + 3 x + 2$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $2 x^{4} + x^{2} + x + 1$ и $x^{4} + 1$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $9272$ делится хотя бы на одно из этих чисел: ${6, 9, 13, 17}$?

## Вариант 26
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(8,25,43,58)$, открытый ключ $(85, 55)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `т`: 13, `а`: 32, `в`: 47, `ш`: 49, `и`: 53, `к`: 54. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 2513674?
1.   Вычислите НОД многочленов $-3 x^{4} + 6 x^{3} + 6 x^{2} - 15 x + 6$ и $6 x^{4} + 15 x^{3} + 12 x^{2} - 9$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $2 x^{4} + x^{3} + 2 x^{2} + 1$ и $2 x^{4} + 2$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $8800$ делится хотя бы на одно из этих чисел: ${13, 14, 15, 16}$?

## Вариант 27
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(78,80,9,43)$, открытый ключ $(85, 47)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `ш`: 1, `е`: 22, `м`: 70, `к`: 72, `т`: 88, `у`: 90. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 6731524?
1.   Вычислите НОД многочленов $6 x^{4} - x^{3} - 11 x^{2} + 4$ и $-6 x^{4} - 8 x^{3} - 7 x^{2} + x + 6$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $x^{4} + x^{3} + x + 1$ и $x^{4} + 2 x^{2} + 2 x + 2$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $8682$ делится хотя бы на одно из этих чисел: ${8, 9, 13, 17}$?

## Вариант 28
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(200,86,59,129)$, открытый ключ $(221, 79)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `к`: 4, `а`: 30, `о`: 52, `м`: 76, `т`: 83, `е`: 92. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 2174356?
1.   Вычислите НОД многочленов $-3 x^{4} + 5 x^{2} + 4 x + 1$ и $3 x^{4} - x^{3} - 4 x^{2} - 3 x - 1$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $x^{4} + 2 x^{2} + x + 2$ и $2 x^{4} + x^{3} + x + 2$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $1164$ делится хотя бы на одно из этих чисел: ${9, 10, 11, 12}$?

## Вариант 29
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(80,30,97,45)$, открытый ключ $(221, 151)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `п`: 10, `с`: 22, `а`: 52, `и`: 55, `ф`: 61, `р`: 80. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 2754136?
1.   Вычислите НОД многочленов $-9 x^{4} - 8 x^{2} - 4 x - 3$ и $-6 x^{4} + 7 x^{3} - 11 x^{2} + 5 x - 3$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $x^{4} + 2 x^{3} + 2 x + 1$ и $x^{4} + x^{2} + 1$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $5900$ делится хотя бы на одно из этих чисел: ${9, 15, 16, 19}$?

## Вариант 30
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(12,152,52,27)$, открытый ключ $(187, 99)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `т`: 6, `а`: 11, `ж`: 40, `к`: 79, `ё`: 93, `с`: 94. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470?
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 3174526?
1.   Вычислите НОД многочленов $-6 x^{4} + 9 x^{3} - 16 x^{2} + 11 x - 6$ и $-6 x^{4} + 3 x^{3} + 8 x^{2} - 11 x + 6$ над $\mathbf{R}[x]$
1.   Вычислите НОД многочленов $2 x^{4} + 2 x^{3} + 2 x^{2} + 1$ и $x^{4} + x^{2} + 1$ над $\mathbf{Z}_3[x]$
1.   Сколько чисел от $1$ до $7419$ делится хотя бы на одно из этих чисел: ${2, 5, 7, 11}$?